In [1]:
import os
# print(os.getcwd())
os.chdir('/pfs/data5/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training')
# print(os.getcwd())

In [2]:
import yaml
import datetime
from logger import create_logger, RankFilter

path = "./config/detector/resnet34_bcos.yaml"
# parse options and load config
with open(path, 'r') as f:
    config = yaml.safe_load(f)
try:# KAI: added this, to ensure it finds the config file
    with open('./training/config/train_config.yaml', 'r') as f:
        config2 = yaml.safe_load(f)
except FileNotFoundError:
    with open(os.path.expanduser('~/Interpretable-Deep-Fake-Detection/training/config/train_config.yaml'), 'r') as f:
        config2 = yaml.safe_load(f)
if 'label_dict' in config:
    config2['label_dict']=config['label_dict']
config.update(config2)
# config['local_rank']=args.local_rank
if config['dry_run']:
    config['nEpochs'] = 0
    config['save_feat']=False


if config['lmdb']:
    config['dataset_json_folder'] = 'preprocessing/dataset_json_v3'
# create logger
timenow=datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
task_str = f"_{config['task_target']}" if config.get('task_target', None) is not None else ""
logger_path =  os.path.join(
            config['log_dir'],
            config['model_name'] + task_str + '_' + timenow
        )
os.makedirs(logger_path, exist_ok=True)
logger = create_logger(os.path.join(logger_path, 'training.log'))
logger.info('Save log to {}'.format(logger_path))
#config['ddp']= args.ddp


# print configuration
logger.info("--------------- Configuration ---------------")
params_string = "Parameters: \n"
for key, value in config.items():
    params_string += "{}: {}".format(key, value) + "\n"
logger.info(params_string)

2025-02-04 19:27:58,773 - INFO - Save log to ./logs/training/resnet34_bcos_2025-02-04-19-27-58
2025-02-04 19:27:58,774 - INFO - --------------- Configuration ---------------
2025-02-04 19:27:58,775 - INFO - Parameters: 
log_dir: ./logs/training/
pretrained: /home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training/pretrained/bcos/resnet34-333f7ec4.pth
from_url: False
model_name: resnet34_bcos
backbone_name: resnet34_bcos
backbone_config: {'num_classes': 2, 'inc': 3, 'dropout': False, 'pretrained': True, 'groups': 1, 'base_width': 64, 'zero_init_residual': False, 'replace_stride_with_dilation': 'None', 'norm_layer': 'None', 'short_cat': False, 'mode': 'Original'}
all_dataset: ['FaceForensics++', 'FF-F2F', 'FF-DF', 'FF-FS', 'FF-NT']
train_dataset: ['FaceForensics++']
test_dataset: ['FaceForensics++']
compression: c40
train_batchSize: 64
test_batchSize: 64
workers: 8
frame_num: {'train': 32, 'test': 32}
resolution: 224
with_mask: False
with_landmark: False
save_ckpt: False
sav

In [3]:
import torch
torch.cuda.empty_cache()
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "" 
sys.argv = ["train.py"]
from train import init_seed, prepare_training_data, prepare_testing_data, choose_optimizer, choose_scheduler, choose_metric
import torch.backends.cudnn as cudnn
import torch.distributed as dist
from datetime import timedelta
from detectors import DETECTOR
from trainer.trainer import Trainer

2025-02-04 19:27:59.177193: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 19:27:59.216812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 19:27:59.216847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 19:27:59.216870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 19:27:59.224923: I tensorflow/core/platform/cpu_feature_g

['in_channels', 'out_channels', 'kernel_size', 'stride', 'padding', 'dilation', 'groups', 'bias', 'padding_mode', 'device', 'dtype']
spatial_count=0 keep_stride_count=0


In [4]:
# init seed
init_seed(config)

# set cudnn benchmark if needed
config['cudnn'] = False
if config['cudnn']:
    cudnn.benchmark = True

config['ddp'] = False
if config['ddp']:
    # dist.init_process_group(backend='gloo')
    dist.init_process_group(
        backend='nccl',
        timeout=timedelta(minutes=30)
    )
    logger.addFilter(RankFilter(0))

# prepare the training data loader
train_data_loader = prepare_training_data(config)

# prepare the testing data loader
test_data_loaders = prepare_testing_data(config)

# prepare the model (detector)
model_class = DETECTOR[config['model_name']]
model = model_class(config)

print(f"Model is on device: {next(model.parameters()).device}")

# prepare the optimizer
optimizer = choose_optimizer(model, config)

# prepare the scheduler
scheduler = choose_scheduler(config, optimizer)

# prepare the metric
metric_scoring = choose_metric(config)

# prepare the trainer
trainer = Trainer(config, model, optimizer, scheduler, logger, metric_scoring, time_now=timenow)
import faulthandler
faulthandler.enable()
# start training
for epoch in range(config['start_epoch'], config['nEpochs'] + 1):
    trainer.model.epoch = epoch
    best_metric = trainer.train_epoch(
                epoch=epoch,
                train_data_loader=train_data_loader,
                test_data_loaders=test_data_loaders,
            )
    if best_metric is not None:
        logger.info(f"===> Epoch[{epoch}] end with testing {metric_scoring}: {parse_metric_for_print(best_metric)}!")
logger.info("Stop Training on best Testing metric {}".format(parse_metric_for_print(best_metric))) 
# # update
# if 'svdd' in config['model_name']:
#     model.update_R(epoch)
# if scheduler is not None:
#     scheduler.step()

# # close the tensorboard writers
# for writer in trainer.writers.values():
#     writer.close()

2025-02-04 19:28:07,123 - INFO - Load pretrained model successfully!
2025-02-04 19:28:07,125 - INFO - Running model on cpu, since device=cpu available.
2025-02-04 19:28:07,126 - INFO - ===> Epoch[0] start!
2025-02-04 19:28:07,168 - INFO - data_dict saved to ./logs/training/resnet34_bcos_2025-02-04-19-27-58/train/FaceForensics++/data_dict_train.pickle


Model is on device: cpu


  0%|          | 0/1793 [00:00<?, ?it/s]

torch.Size([64, 2])
torch.Size([64])


  0%|          | 0/1793 [00:35<?, ?it/s]


KeyError: 'local_rank'

In [ ]:
!squeue -l
!scontrol show job 25223887    
!sstat -j 25220981 --format=JobID,MaxRSS,AveRSS,AveCPU
!free -h  # Memory usage summary
!srun --jobid=25223887 nvidia-smi

In [ ]:
!kill 2412

In [ ]:
from b_cos.resnet import resnet50
from b_cos.bcosconv2d import BcosConv2d
import torch
import torch.nn as nn
import torch.optim as optim
from torch.hub import load_state_dict_from_url
import matplotlib.pyplot as plt
import torchvision.models as models
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torchvision import transforms, datasets
from tqdm import tqdm
from PIL import Image
import os
import numpy as np
import torch.nn.functional as F

In [ ]:
class MyToTensor(transforms.ToTensor):
    def __init__(self):
        """
        Same as transforms.ToTensor, except that if input to __call__ is already tensor, the input is returned unchanged
        """
        super().__init__()

    def __call__(self, input_img):
        if not isinstance(input_img, torch.Tensor):
            return super().__call__(input_img)
        return input_img

class AddInverse(nn.Module):
    def __init__(self, dim=1):
        """
            Adds (1-in_tensor) as additional channels to its input via torch.cat().
            Can be used for images to give all spatial locations the same sum over the channels to reduce color bias.
        """
        super().__init__()
        self.dim = dim

    def forward(self, in_tensor):
        out = torch.cat([in_tensor, 1-in_tensor], self.dim)
        return out
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize if necessary
    MyToTensor(),            # Converts image to tensor if not already
    AddInverse(dim=0),            # Adds the inverse channels
])

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, folder_paths, transform=None):
        """
        Args:
            folder_paths (dict): Dictionary where keys are folder paths, and values are the labels.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.image_files = []  # Store (image_path, label) tuples
        for fp, label in folder_paths.items():
            # List all subdirectories within the folder
            subfolders = [os.path.join(fp, d) for d in os.listdir(fp) if os.path.isdir(os.path.join(fp, d))]
            for folder_path in subfolders:
                # Collect all image paths from the folder and assign the given label
                self.image_files.extend(
                    [(os.path.join(folder_path, f), label) for f in os.listdir(folder_path) if f.endswith((".png", ".jpg"))]
                )
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, label = self.image_files[idx]  # Get image path and its label
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


file_path_deepfakebench = {'../../DeepfakeBench/datasets/rgb/FaceForensics++/manipulated_sequences/DeepFakeDetection/c40/frames': 1,
                           "../../DeepfakeBench/datasets/rgb/FaceForensics++/original_sequences/actors/c40/frames": 0}

dataset = CustomImageDataset(file_path_deepfakebench, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for images, label in dataloader:
    print(f"Batch of images shape: {images.shape}")
    break

In [ ]:
def load_pretrained_weights(model, model_url):
    # Load the state dict from the URL
    state_dict = load_state_dict_from_url(model_url, progress=True)

    # Rename keys to match custom model layer names and initialize adapted state dict
    adapted_state_dict = {}
    for key, value in state_dict.items():
        # Replace ".weight" with ".linear.weight" if necessary for `BcosConv2d`
        new_key = key.replace("conv", "conv.linear").replace("fc", "fc.linear")
        
        # Only add weights if the shape matches the model's layer
        if new_key in model.state_dict() and model.state_dict()[new_key].shape == value.shape:
            adapted_state_dict[new_key] = value

    # Load the adapted state dict with partial loading (non-strict mode)
    model.load_state_dict(adapted_state_dict, strict=False)

    # Reinitialize the final layer to match the desired output shape (for 2 classes)
    nn.init.kaiming_normal_(model.fc.linear.weight)
    if model.fc.linear.bias is not None:
        model.fc.linear.bias.data.zero_()  # Initialize bias to zero if it exists

# Usage with resnet50
model = resnet50(pretrained=False, progress=True, num_classes=1, groups=32, width_per_group=4)
load_pretrained_weights(model, 'https://download.pytorch.org/models/resnet50-19c8e357.pth')

In [ ]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss() #nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1
fraction = 1
num_batches = int(len(dataloader) * fraction)

# Training loop with limited batches
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(dataloader, total=num_batches, desc=f'Epoch [{epoch + 1}/{num_epochs}]')
    for batch_idx, (images, labels) in enumerate(progress_bar):
        if batch_idx >= num_batches:
            break  # Stop after reaching the fraction of batches

        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        # outputs = outputs.view(outputs.size(0), -1) NEEDED WHEN USING CROSS ENTROPY LOSS & num_classes > 1

        # Convert labels to float for BCEWithLogitsLoss
        labels = labels.float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        progress_bar.set_postfix({'Loss': running_loss / (batch_idx + 1)})
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / num_batches:.4f}')

In [ ]:
torch.save(model.state_dict(), 'b_cos_model_2.pth')

In [ ]:
model_trained = resnet50(pretrained=False, progress=True, num_classes=1, groups=32, width_per_group=4)

model_trained.load_state_dict(torch.load('b_cos_model_x.pth', map_location=torch.device('cpu')))

## Heatmaps

In [ ]:
def to_numpy(tensor):
    """
    Converting tensor to numpy.
    Args:
        tensor: torch.Tensor

    Returns:
        Tensor converted to numpy.

    """
    if not isinstance(tensor, torch.Tensor):
        return tensor
    return tensor.detach().cpu().numpy()

def grad_to_img(img, linear_mapping, smooth=15, alpha_percentile=99.5):
    """
    Computing color image from dynamic linear mapping of B-cos models.
    Args:
        img: Original input image (encoded with 6 color channels)
        linear_mapping: linear mapping W_{1\rightarrow l} of the B-cos model
        smooth: kernel size for smoothing the alpha values
        alpha_percentile: cut-off percentile for the alpha value

    Returns:
        image explanation of the B-cos model
    """
    # shape of img and linmap is [C, H, W], summing over first dimension gives the contribution map per location
    contribs = (img * linear_mapping).sum(0, keepdim=True)
    contribs = contribs[0]
    # Normalise each pixel vector (r, g, b, 1-r, 1-g, 1-b) s.t. max entry is 1, maintaining direction
    rgb_grad = (linear_mapping / (linear_mapping.abs().max(0, keepdim=True)[0] + 1e-12))
    # clip off values below 0 (i.e., set negatively weighted channels to 0 weighting)
    rgb_grad = rgb_grad.clamp(0)
    # normalise s.t. each pair (e.g., r and 1-r) sums to 1 and only use resulting rgb values
    rgb_grad = to_numpy(rgb_grad[:3] / (rgb_grad[:3] + rgb_grad[3:]+1e-12))

    # Set alpha value to the strength (L2 norm) of each location's gradient
    alpha = (linear_mapping.norm(p=2, dim=0, keepdim=True))
    # Only show positive contributions
    alpha = torch.where(contribs[None] < 0, torch.zeros_like(alpha) + 1e-12, alpha)
    if smooth:
        alpha = F.avg_pool2d(alpha, smooth, stride=1, padding=(smooth-1)//2)
    alpha = to_numpy(alpha)
    alpha = (alpha / np.percentile(alpha, alpha_percentile)).clip(0, 1)

    rgb_grad = np.concatenate([rgb_grad, alpha], axis=0)
    # Reshaping to [H, W, C]
    grad_image = rgb_grad.transpose((1, 2, 0))
    return grad_image

In [ ]:
for img_batch, label_batch in dataloader:
    print(f"Batch of images shape: {img_batch.shape}")
    device = "cpu"
    
    # Iterate through each image in the batch
    for i in range(len(img_batch)):
        img = img_batch[i].unsqueeze(0).to(device)  # Process a single image
        label = label_batch[i]

        img = img.requires_grad_(True)

        # Zero the gradients
        model_trained.zero_grad()

        # Forward pass
        out = model_trained(img)

        # Backward pass
        out.backward()

        # Generate attention visualization
        att = grad_to_img(img[0], img.grad[0], alpha_percentile=100, smooth=5)
        att[..., -1] *= to_numpy(out.sigmoid())

        # Prepare the image and attention map for visualization
        att = to_numpy(att)
        img_np = np.array(to_numpy(img[0, :3].permute(1, 2, 0)) * 255, dtype=np.uint8)

        # Plot the results
        fig, ax = plt.subplots(1, figsize=(8, 4))
        plt.imshow(img_np, extent=(0, 224, 0, 224))
        plt.imshow(att, extent=(224, 2 * 224, 0, 224))
        plt.xlim(0, 2 * 224)
        plt.xticks([])
        plt.yticks([])
        plt.title(f"True Label: {label}, Predictions: {out.sigmoid().item():.2f}")
        for spine in ax.spines.values():
            spine.set_visible(False)

        plt.show()

    break  # Exit after processing the first batch


In [ ]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_trained.to(device)

# Set model_trained to evaluation mode
model_trained.eval()

# Lists to store all predictions and true labels
all_preds = []
all_labels = []

# Disable gradient calculation
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model_trained(images)  # Forward pass
        outputs = outputs.view(outputs.size(0), -1)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        all_preds.extend(predicted.cpu().numpy())  # Store predictions
        all_labels.extend(labels.cpu().numpy())  # Store true labels

# Calculate and print the accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Test Accuracy: {accuracy:.2f}')

# Detailed classification report
print(classification_report(all_labels, all_preds, target_names=['Fake', 'Real']))
print(confusion_matrix(all_labels, all_preds))